In [1]:
# Import libraries
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import itertools
import argparse
import sys
import time
from sklearn import preprocessing
import pandas as pd
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import random
from math import floor

In [2]:
## Preprocessing of data
# Function to load data

def get_power_data():
    """
    Read the Individual household electric power consumption dataset
    """
    
    # Assume that the dataset is located on folder "data"
    data = pd.read_csv('data/household_power_consumption.txt',
                       sep=';', low_memory=False)

    # Drop some non-predictive variables
    data = data.drop(columns=['Date', 'Time'], axis=1)

    #print(data.head())

    # Replace missing values
    data = data.replace('?', np.nan)

    # Drop NA
    data = data.dropna(axis=0)

    # Normalize
    standard_scaler = preprocessing.StandardScaler()
    np_scaled = standard_scaler.fit_transform(data)
    data = pd.DataFrame(np_scaled)

    # Goal variable assumed to be the first
    X = data.values[:, 1:].astype('float32')
    y = data.values[:, 0].astype('float32')

    # Create categorical y for binary classification with balanced classes
    y = np.sign(y+0.46)

    # Split train and test data here: (X_train, Y_train, X_test, Y_test)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    no_class = 2                 #binary classification

    return X_train.T, X_test.T, y_train, y_test, no_class

In [3]:
X_train, X_test, y_train, y_test, no_class = get_power_data()
print("X,y types: {} {}".format(type(X_train), type(y_train)))
print("X size {}".format(X_train.shape))
print("Y size {}".format(y_train.shape))

# Create a binary variable from one of the columns.
# You can use this OR not

idx = y_train >= 0
notidx = y_train < 0
y_train[idx] = 1
y_train[notidx] = -1

X,y types: <class 'numpy.ndarray'> <class 'numpy.ndarray'>
X size (6, 1536960)
Y size (1536960,)


Let
\begin{align}
    E=\min_{w_3, W_2,W_1} \frac{1}{N}\sum_i || w_3 s(W_2 s(W_1 x_i)-y_i)||^2   
\end{align}

## Layer 3
Define 
\begin{align}
a_3(x)& :=w_3 s(W_2 s(W_1 x))\\
a_2(x)& :=s(W_2 s(W_1 x)),\\
a_1(x)& :=s(W_1 x).
\end{align}

Then
\begin{align}
\frac{\partial E}{\partial w_3} &= \frac{2}{N}(a_3-t)\frac{\partial a_3}{\partial w_3} \\
&=\frac{2}{N}(x_3-t)\frac{\partial w_3a_2}{\partial w_3}\\
&=\frac{2}{N}(x_3-t)a_2^T\\
\end{align}

So defining 
$$\delta_3 := \frac{2}{N}(a_3-t),$$
then
$$\frac{\partial E}{\partial w_3} =\delta_3\,a_2^T.$$

## Layer 2

\begin{align*}
\frac{\partial E}{\partial W_2} &= \frac{2}{N}(a_3-t)\frac{\partial a_3}{\partial W_2} \\
&=\frac{2}{N}(a_3-t)\frac{\partial (W_3 a_2)}{\partial W_2}\\
&=\delta_3\frac{\partial (W_3 a_2)}{\partial W_2}\\
&=W_3^T\delta_3\frac{\partial a_2}{\partial W_2}\\
&=[W_3^T\delta_3 \circ s'(W_2 a_1)]\frac{\partial W_2 a_1}{\partial W_2}\\
\end{align*}

So defining $$\delta_2 :=W_3^T\delta_3 \circ s'(W_2 a_1),$$
we have

$$\frac{\partial E}{\partial W_2}=\delta_2 a_1^T$$

## Layer 1

Define 
$$\delta_1 :=W_2^T\delta_2 \circ s'(W_1x),$$
similar to layer_2:
\begin{align*}
\frac{\partial E}{\partial W_1} &=\delta_1x^T
\end{align*}

In [10]:
# Sigmoid function
def sigmoid(x, derivative=False):
    sigm = 1. / (1. + np.exp(-x))
    if derivative:
        return sigm * (1. - sigm)
    return sigm

# Define weights initialization
def initialize_w(N, d):
    return 2*np.random.random((N,d)) - 1

# Fill in feed forward propagation
def feed_forward_propagation(X, y, w_1, w_2, w_3, lmbda):
    # Fill in
    #X is q x n
    # w_1 is p x q
    # w_2 is p x p
    # w_3 is 1 x p
    layer_0=X # q x n
    layer_1=sigmoid(np.matmul(w_1 , X)) # p x n 
    layer_2=sigmoid(np.matmul(w_2 , layer_1)) # p x n 
    layer_3=np.matmul(w_3 , layer_2) # p x n
    return layer_0, layer_1, layer_2, layer_3


# Fill in backpropagation    
def back_propagation(y, w_1, w_2, w_3, layer_0, layer_1, layer_2, layer_3):
    # Calculate the gradient here
    N = y.shape[0]    
        
    delta3=2/N*(layer_3 - y)
    delta2=np.multiply(np.matmul(w_3.T,delta3),sigmoid(np.matmul(w_2,layer_1),derivative=True))
    delta1=np.multiply(np.matmul(w_2.T,delta2),sigmoid(np.matmul(w_1,layer_0),derivative=True))
    
    layer_3_delta=np.matmul(delta3,layer_2.T)
    layer_2_delta=np.matmul(delta2,layer_1.T)
    layer_1_delta=np.matmul(delta1,layer_0.T)

    return layer_1_delta, layer_2_delta, layer_3_delta


# Cost function
def cost(X, y, w_1, w_2, w_3, lmbda):
    N, d = X.shape
    a1,a2,a3,a4 = feed_forward_propagation(X,y,w_1,w_2,w_3,lmbda)

    return np.linalg.norm(a4[:,0] - y,2) ** 2 / N

# Funtion to get mini batch sgd
def miniBatch(x,y,batchSize):
    D,N = x.shape
    X_mini = np.zeros((D,batchSize))
    Y_mini = np.zeros((batchSize,))
    indexArray = random.sample(range(N), batchSize)
    for i in range(batchSize):
        X_mini[:,i] = x[:,indexArray[i]]
        Y_mini[i,] = y[indexArray[i],]
    return X_mini,Y_mini

# Define SGD
def SGD(X, y, w_1, w_2, w_3, lmbda, learning_rate, batch_size, iterations):
    cost_l=[]
    for i in range(iterations):

        X_mini,Y_mini = miniBatch(X,y,batch_size)
        L0,L1,L2,L3 = feed_forward_propagation(X_mini,Y_mini,w_1,w_2,w_3,lmbda)
        D1,D2,D3 = back_propagation(Y_mini,w_1,w_2,w_3,L0,L1,L2,L3)

        #cost1 = cost(X_mini, Y_mini, w_1, w_2, w_3, lmbda)
        
        a = w_1-(learning_rate*D1).reshape(w_1.shape)
        b = w_2-(learning_rate*D2).reshape(w_2.shape)
        c = w_3-(learning_rate*D3).reshape(w_3.shape)
        
        #cost2 = cost(X_mini, Y_mini, a, b, c, lmbda)
    
        #if ((cost2-cost1)/cost1>0.5):
        #    break
        
        w_1 = a
        w_2 = b
        w_3 = c
                
        cost_l.append(cost(X,y,w_1,w_2,w_3,lmbda=lmbda))
        #print(i,': ', cost_l[-1])
    return w_1, w_2, w_3, cost_l

# Define SVRG here:
def SVRG(X, y, w_1, w_2, w_3, lmbda, learning_rate, T,M,iterations):
    #M is the numebr of samples used in the minibatch
    cost_l=[]    
    for i in range(iterations):
        
        K  = floor(iterations/T)
        N  = X.shape[1]
        wk_1= w_1
        wk_2= w_2
        wk_3= w_3
        
        for k in range(K):
            L0,L1,L2,L3 = feed_forward_propagation(X,y,w_1,w_2,w_3,lmbda)
            ga_1, ga_2, ga_3 = back_propagation(y,wk_1,wk_2,wk_3,L0,L1,L2,L3) #the average
            
            for t in range(T):
                index = np.random.randint(N, size=M)
                L0,L1,L2,L3 = feed_forward_propagation(X[:,index],y[index,],w_1,w_2,w_3,lmbda)
                g1_1,g1_2,g1_3 = back_propagation(y[index,], w_1,w_2,w_3,L0,L1,L2,L3)
                
                Lk0,Lk1,Lk2,Lk3 = feed_forward_propagation(X[:,index],y[index,],wk_1,wk_2,wk_3,lmbda)
                g2_1,g2_2,g2_3 = back_propagation(y[index,], wk_1,wk_2,wk_3,Lk0,Lk1,Lk2,Lk3)
                
                g1  = g1_1 - g2_1 + ga_1
                g2  = g1_2 - g2_2 + ga_2
                g3  = g1_3 - g2_3 + ga_3

                #cost1 = cost(X, y, w_1, w_2, w_3, lmbda)
            
                w_1 = w_1 - (learning_rate*g1).reshape(w_1.shape)
                w_2 = w_2 - (learning_rate*g2).reshape(w_2.shape)
                w_3 = w_3 - (learning_rate*g3).reshape(w_3.shape)
            

            wk_1 = w_1
            wk_2 = w_2
            wk_3 = w_3
        cost_l.append(cost(X,y,w_1,w_2,w_3,lmbda=lmbda))    
        #print(i,': ', cost_l[-1])        
    return w_1, w_2, w_3, cost_l

# Define GD here:
def GD(X, y, w_1,w_2,w_3, learning_rate, lmbda, iterations):
    cost_l=[]
    for i in range(iterations): 
        
        L0,L1,L2,L3 = feed_forward_propagation(X,y,w_1,w_2,w_3,lmbda)
        D1,D2,D3 = back_propagation(y,w_1,w_2,w_3,L0,L1,L2,L3)
    
        #cost1 = cost(X, y, w_1, w_2, w_3, lmbda)
        
        w_1 = w_1-(learning_rate*D1).reshape(w_1.shape)
        w_2 = w_2-(learning_rate*D2).reshape(w_2.shape)
        w_3 = w_3-(learning_rate*D3).reshape(w_3.shape)
        
        cost_l.append(cost(X,y,w_1,w_2,w_3,lmbda=lmbda))
        #print(i,': ', cost_l[-1])        
    
    return w_1, w_2, w_3, cost_l

# Define projected GD here:
def PGD(X, y, w_1,w_2,w_3, learning_rate, lmbda, iterations, noise):
    # Complete here:
    
    return w_1, w_2, w_3

# Define BCD here:
def BCD(X, y, w_1,w_2,w_3, learning_rate, lmbda, iterations):
    # Complete here:
    
    return w_1, w_2, w_3







In [ ]:
w_1 = initialize_w(3,X_train.shape[0])
w_2 = initialize_w(3,3)
w_3 = initialize_w(1,3)
lmbda=0.1
print(w_1.shape)
print(w_2.shape)
print(w_3.shape)

print(X_train.shape)
print(y_train.shape)

initialCost=cost(X_train,y_train,w_1,w_2,w_3,lmbda)
layer_0,layer_1,layer_2,layer_3 = feed_forward_propagation(X_train,y_train,w_1,w_2,w_3,lmbda)
#print('cost: ',costx)

w_1_GD,w_2_GD,w_3_GD,cost_l = GD(X_train, y_train, w_1,w_2,w_3, learning_rate = 0.1, lmbda=lmbda, iterations=10)
finalCost=cost(X_train,y_train,w_1_GD,w_2_GD,w_3_GD,lmbda)
print('Initial Cost:',initialCost)
print('Initial Cost:',finalCost)
#d1,d2,d3 = back_propagation(y_train, w_1, w_2, w_3, layer_0,layer_1,layer_2,layer_3)

In [ ]:
w_1 = initialize_w(3,X_train.shape[0])
w_2 = initialize_w(3,3)
w_3 = initialize_w(1,3)
lmbda=0.1
#SGD(X, y, w_1, w_2, w_3, lmbda, learning_rate, batch_size, iterations):
initialCost = cost(X_train,y_train,w_1,w_2,w_3,lmbda=0.1)
w_1_SGD,w_2_SGD,w_3_SGD,cost_l = SGD(X_train, y_train, w_1, w_2, w_3, lmbda=0.1, learning_rate=0.05,batch_size=100,iterations=20)
finalCost = cost(X_train,y_train,w_1_SGD,w_2_SGD,w_3_SGD,lmbda=0.1)
print('Initial cost:',initialCost)
print('Final cost:',finalCost)



In [ ]:
w_1 = initialize_w(3,X_train.shape[0])
w_2 = initialize_w(3,3)
w_3 = initialize_w(1,3)
initialCost = cost(X_train,y_train,w_1,w_2,w_3,lmbda=0.1)
print('Initial cost:',initialCost)
s1,s2,s3 = SVRG(X_train, y_train, w_1, w_2, w_3, lmbda=0.1, learning_rate=0.05,T=2,M=100,iterations=10)
finalCost = cost(X_train,y_train,s1,s2,s3,lmbda=0.1)
print('Initial cost:',initialCost)
print('Final cost:',finalCost)

In [19]:
# Tuning hyper parameters:
GD_params=[]
GD_cost=[]

SGD_params=[]
SGD_cost=[]

SVRG_params=[]
SVRG_cost=[]

for W_SIZE in [3,5,10]:
    while(True):
        w_1 = initialize_w(W_SIZE,X_train.shape[0])
        w_2 = initialize_w(W_SIZE,W_SIZE)
        w_3 = initialize_w(1,W_SIZE)    
        initialCost = cost(X_train,y_train,w_1,w_2,w_3,lmbda=0.1)
        if(initialCost>300000):
            print("init:",initialCost)
            break
    
    
    for LAMBDA in [0.01,0.05,0.1]:
        for LR in [0.01,0.05,0.1]:
            
            #GD
            GD_params.append([W_SIZE,LAMBDA,LR])
            w_1_GD,w_2_GD,w_3_GD,cost_l_GD = GD(X_train, y_train, w_1,w_2,w_3, learning_rate = LR, lmbda=LAMBDA, iterations=50)
            GD_cost.append(cost_l_GD)
            print("GD:",GD_cost[-1][-1])
           
            #SGD
            for B_SIZE in [2,10,100]:
                SGD_params.append([W_SIZE,LAMBDA,LR,B_SIZE])
                w_1_SGD,w_2_SGD,w_3_SGD,cost_l_SGD = SGD(X_train, y_train, w_1,w_2,w_3, learning_rate = LR, lmbda=LAMBDA,batch_size=B_SIZE, iterations=50)
                SGD_cost.append(cost_l_SGD)
                print("SGD:",SGD_cost[-1][-1])   

            #SVRG
            for T_ in [2,5,10]:
                for M_ in [2,10,100]:
                    SVRG_params.append([W_SIZE,LAMBDA,LR,T_,M_])
                    w_1_SVRG,w_2_SVRG,w_3_SVRG,cost_l_SVRG = SVRG(X_train, y_train, w_1,w_2,w_3, learning_rate = LR, lmbda=LAMBDA,T=T_,M=M_, iterations=10)
                    SVRG_cost.append(cost_l_SVRG)
                    print("SVRG:",SVRG_cost[-1][-1])                   
                
            
            
        
        



init: 593458.257628264
GD: 330177.1493233919
SGD: 347599.5229124606
SGD: 315453.38908577844
SGD: 335218.74402868573
SVRG: 272895.24425845064
SVRG: 272897.9345898819
SVRG: 272900.86997842096
SVRG: 272900.489568686
SVRG: 272907.1906321781
SVRG: 272900.4409631461
SVRG: 272968.2296752951
SVRG: 272915.7694902402
SVRG: 272899.80859681574
GD: 256174.73857817857
SGD: 256159.92059735706
SGD: 256463.3554553511
SGD: 256177.6200538317
SVRG: 256339.90861288874
SVRG: 256339.62635491753
SVRG: 256339.66355080425
SVRG: 256335.5450042203
SVRG: 256340.90691920547
SVRG: 256338.74372062043
SVRG: 256334.92999192805
SVRG: 256338.8126332441
SVRG: 256339.96085549463
GD: 256344.10816724575
SGD: 257446.7989660696
SGD: 256566.9924871412
SGD: 256166.17386268298
SVRG: 256623.01092948936
SVRG: 256618.4432466093
SVRG: 256618.41552125066
SVRG: 256615.06833699936
SVRG: 256621.42183625372
SVRG: 256618.86259861043
SVRG: 256643.8932451836
SVRG: 256598.3398834743
SVRG: 256619.44807474685
GD: 330177.1493233919
SGD: 299289.5

SGD: 364509.79562404874
SGD: 377255.0815711035
SGD: 357885.0508183019
SVRG: 377215.94398201205
SVRG: 377602.1987036252
SVRG: 377436.09200918017
SVRG: 379070.73531186796
SVRG: 376635.23746871296
SVRG: 377262.6497916462
SVRG: 383021.48898906494
SVRG: 376781.954849212
SVRG: 377093.8504442163


NameError: name 'cost_GD' is not defined

In [23]:
import pickle as pk

# open a file, where you ant to store the data
file = open('Hyper', 'wb')
# dump information to that file
pk.dump((GD_cost,GD_params,SGD_cost,SGD_params,SVRG_cost,SVRG_params), file)

file.close()

In [22]:
SVRG_params=[]
for W_SIZE in [3,5,10]:

    for LAMBDA in [0.01,0.05,0.1]:
        for LR in [0.01,0.05,0.1]:

            #SVRG
            for T_ in [2,5,10]:
                for M_ in [2,10,100]:
                    SVRG_params.append([W_SIZE,LAMBDA,LR,T_,M_])
    